In [13]:
import os
import httpx
from pathlib import Path
from dotenv import load_dotenv
from tools import *
from openai import OpenAI
load_dotenv()

client = OpenAI(
    base_url="https://api.moonshot.cn/v1",
    api_key=os.environ.get("Kimi_API_KEY"),
)

In [5]:
text = file_extraction("test files/象上-文稿-转写结果.docx")

In [16]:
def upload_files(files, cache_tag):
    messages = []

    for file in files:
        file_object = client.files.create(file=Path(file), purpose="file-extract")
        file_content = client.files.content(file_id=file_object.id).text
        messages.append({
            "role": "system",
            "content": file_content,
        })

    if cache_tag:
        r = httpx.post(f"{client.base_url}/caching",
                       headers={
                           "Authorization": f"Bearer {client.api_key}",
                       },
                       json={
                           "model": "moonshot-v1",
                           "messages": messages,
                           "ttl": 300,
                           "tags": [cache_tag],
                       })

        return [{
            "role": "cache",
            "content": f"tag={cache_tag};reset_ttl=300",
        }]
    else:
        return messages

In [17]:
k = upload_files(['test files/象上-文稿-转写结果.docx'], "text")

In [18]:
k

[{'role': 'cache', 'content': 'tag=text;reset_ttl=300'}]

In [19]:
def generate_summary_from_cache(cache_tag: str) -> str:
    messages = [
        {
            "role": "cache",
            "content": f"tag={cache_tag};reset_ttl=300",
        },
        {
            "role": "system",
            "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，"
                       "准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不"
                       "可翻译成其他语言。",
        },
        {
            "role": "user",
            "content": "总结一下这些文件的内容。",
        },
    ]

    completion = client.chat.completions.create(
        model="moonshot-v1-128k",
        messages=messages,
    )

    return completion.choices[0].message.content

# 上传文件并创建缓存
cache_tag = "text"
upload_files(
    files=["test files/象上-文稿-转写结果.docx"],
    cache_tag=cache_tag,
)

# 访问缓存并生成总结
summary = generate_summary_from_cache(cache_tag)
print("总结：", summary)

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Permission denied, not allowed to access the feature', 'type': 'permission_denied_error'}}

In [ ]:
test_chunks = ['']